### load data

In [1]:
scene = "football"

In [ ]:
import numpy as np 

point_tracks = np.load(f"/restricted/projectnb/cs599dg/mwakeham/datasets/panoptic-multiview/{scene}/tapvid3d_annotations.npz")

print(list(point_tracks.keys()))
print(point_tracks['trajectories'].shape)
print(point_tracks['trajectories_pixelspace'].shape)
print(point_tracks['per_view_visibilities'].shape)
print(point_tracks['intrinsics'].shape)
print(point_tracks['extrinsics'].shape)

### get examples

sample 4 views

sample 200 3D point tracks

keep the 100 with most visibility across the views

In [ ]:
import numpy as np

np.random.seed(1234)

visibilities = point_tracks['per_view_visibilities']
pixel_trajectories = point_tracks['trajectories_pixelspace']

# basketball scene
num_repeats = 100
num_views = 1
num_candidate_points = 2000
num_keep_points = 2000
min_frames_per_view = 0
max_attempts = 1000000

# # boxes scene
# num_repeats = 100
# num_views = 4
# num_candidate_points = 500
# num_keep_points = 100
# min_frames_per_view = 130
# max_attempts = 1000000

# # football scene
# num_repeats = 100
# num_views = 4
# num_candidate_points = 500
# num_keep_points = 100
# min_frames_per_view = 115
# max_attempts = 1000000

# # juggle scene
# num_repeats = 100
# num_views = 4
# num_candidate_points = 500
# num_keep_points = 100
# min_frames_per_view = 100
# max_attempts = 1000000

# # softball scene
# num_repeats = 100
# num_views = 4
# num_candidate_points = 500
# num_keep_points = 100
# min_frames_per_view = 90
# max_attempts = 1000000

# # tennis scene
# num_repeats = 100
# num_views = 4
# num_candidate_points = 500
# num_keep_points = 100
# min_frames_per_view = 100
# max_attempts = 1000000

all_candidate_vis_counts = []
all_selected_vis_counts = []

all_examples = []
attempts = 0

while len(all_examples) < num_repeats and attempts < max_attempts:
    attempts += 1
    
    selected_views = np.random.choice(visibilities.shape[0], size=num_views, replace=False)
    
    candidate_points = np.random.choice(visibilities.shape[2], size=num_candidate_points, replace=False)
    
    vis = visibilities[selected_views][:, :, candidate_points]
    
    frames_visible_per_view = (vis >= 1).sum(axis=1)
    
    valid_points_mask = np.all(frames_visible_per_view >= min_frames_per_view, axis=0)
    valid_candidate_points = candidate_points[valid_points_mask]
    
    if len(valid_candidate_points) < num_keep_points:
        continue
    
    total_vis = vis[:, :, valid_points_mask].sum(axis=0).sum(axis=0)
    top_points_idx = np.argsort(-total_vis)[:num_keep_points]
    selected_points = valid_candidate_points[top_points_idx]
    
    all_candidate_vis_counts.extend(vis.sum(axis=0).sum(axis=0))

    selected_mask = np.isin(candidate_points, selected_points)
    all_selected_vis_counts.extend(vis[:, :, selected_mask].sum(axis=0).sum(axis=0))
    
    traj_pixel = pixel_trajectories[selected_views][:, :, selected_points, :]
    
    all_examples.append({
        'views': selected_views,
        'points': selected_points,
        'trajectories_pixel': traj_pixel
    })
    
    if len(all_examples) % 10 == 0:
        print(f"Collected {len(all_examples)}/{num_repeats} examples (attempt {attempts})")

if all_examples:
    avg_vis_candidates = np.mean(all_candidate_vis_counts)
    sd_vis_candidates = np.std(all_candidate_vis_counts)
    avg_vis_selected = np.mean(all_selected_vis_counts)
    sd_vis_selected = np.std(all_selected_vis_counts)

    print(f"\nSuccessfully collected {len(all_examples)} examples")
    print(f"Success rate: {len(all_examples)/attempts*100:.1f}%")
    print(f"Candidate points visibility: avg={avg_vis_candidates:.2f}, sd={sd_vis_candidates:.2f}")
    print(f"Selected points visibility: avg={avg_vis_selected:.2f}, sd={sd_vis_selected:.2f}")
else:
    print("No examples met the criteria! You may need to relax min_frames_per_view")

### save visualization videos

In [ ]:
import os
import subprocess
import cv2
import numpy as np
import tempfile

base_path = f"/restricted/projectnb/cs599dg/mwakeham/datasets/panoptic-multiview/{scene}/ims"

i = 99 # example number
example = all_examples[i]
views_to_convert = example['views']
trajectories_pixel = example['trajectories_pixel']

visibilities = point_tracks['per_view_visibilities']
selected_points = example['points']
selected_views = example['views']

point_visibilities = visibilities[selected_views][:, :, selected_points]

print(f"Converting views: {views_to_convert}")

fade_frames = 10
color2 = (180, 105, 255)

for view_idx, view in enumerate(views_to_convert):
    image_folder = os.path.join(base_path, str(view))
    output_dir = f"examples_visualization/{scene}/example{i}"
    os.makedirs(output_dir, exist_ok=True)
    
    import glob
    image_files = sorted(glob.glob(os.path.join(image_folder, "*.jpg")))
    
    if not image_files:
        print(f"No images found in {image_folder}")
        continue
    
    with tempfile.TemporaryDirectory() as temp_dir_original, \
         tempfile.TemporaryDirectory() as temp_dir_overlay, \
         tempfile.TemporaryDirectory() as temp_dir_tracks_only:
        
        print(f"Processing view {view}...")
        
        view_trajectories = trajectories_pixel[view_idx]
        view_visibilities = point_visibilities[view_idx]
        num_frames = len(view_trajectories)
        
        for frame_idx, image_path in enumerate(image_files):
            if frame_idx >= num_frames:
                break
                
            img = cv2.imread(image_path)
            if img is None:
                continue
            
            original_frame_path = os.path.join(temp_dir_original, f"frame_{frame_idx:06d}.jpg")
            cv2.imwrite(original_frame_path, img)
            
            img_with_overlay = img.copy()
            
            tracks_only = np.zeros_like(img)
            
            for point_idx in range(len(view_trajectories[frame_idx])):
                if view_visibilities[frame_idx, point_idx] < 1:
                    continue
                
                current_x, current_y = view_trajectories[frame_idx, point_idx]
                if np.isnan(current_x) or np.isnan(current_y):
                    continue
                
                current_x_int, current_y_int = int(round(current_x)), int(round(current_y))
                
                start_frame = max(0, frame_idx - fade_frames)
                
                for hist_frame in range(start_frame, frame_idx):
                    if (view_visibilities[hist_frame, point_idx] < 1 or 
                        view_visibilities[hist_frame + 1, point_idx] < 1):
                        continue
                    
                    fade_idx = frame_idx - hist_frame - 1
                    if fade_idx >= fade_frames:
                        continue
                    
                    ratio = fade_idx / fade_frames
                    b = int(255 + (180 - 255) * ratio)
                    g = int(0 + (105 - 0) * ratio)
                    r = int(255 + (255 - 255) * ratio)
                    color = (b, g, r)
                    
                    hist_x, hist_y = view_trajectories[hist_frame, point_idx]
                    next_x, next_y = view_trajectories[hist_frame + 1, point_idx]
                    
                    if (not np.isnan(hist_x) and not np.isnan(hist_y) and 
                        not np.isnan(next_x) and not np.isnan(next_y)):
                        
                        hist_x_int, hist_y_int = int(round(hist_x)), int(round(hist_y))
                        next_x_int, next_y_int = int(round(next_x)), int(round(next_y))
                        
                        cv2.line(img_with_overlay, (hist_x_int, hist_y_int), (next_x_int, next_y_int), color, 1)
                        cv2.line(tracks_only, (hist_x_int, hist_y_int), (next_x_int, next_y_int), color, 1)
                
                cv2.circle(img_with_overlay, (current_x_int, current_y_int), 1, color2, -1)
                cv2.circle(tracks_only, (current_x_int, current_y_int), 1, color2, -1)
            
            overlay_frame_path = os.path.join(temp_dir_overlay, f"frame_{frame_idx:06d}.jpg")
            cv2.imwrite(overlay_frame_path, img_with_overlay)
            
            tracks_frame_path = os.path.join(temp_dir_tracks_only, f"frame_{frame_idx:06d}.jpg")
            cv2.imwrite(tracks_frame_path, tracks_only)
        
        # Original video
        original_output = os.path.join(output_dir, f"view{view}.mp4")
        cmd_original = [
            'ffmpeg', 
            '-framerate', '30',
            '-i', os.path.join(temp_dir_original, 'frame_%06d.jpg'),
            '-c:v', 'libx264',
            '-pix_fmt', 'yuv420p',
            original_output,
            '-y'
        ]
        
        # Video with tracks overlay
        overlay_output = os.path.join(output_dir, f"view{view}_with_tracks.mp4")
        cmd_overlay = [
            'ffmpeg', 
            '-framerate', '30',
            '-i', os.path.join(temp_dir_overlay, 'frame_%06d.jpg'),
            '-c:v', 'libx264',
            '-pix_fmt', 'yuv420p',
            overlay_output,
            '-y'
        ]
        
        # Tracks only on black background
        tracks_output = os.path.join(output_dir, f"tracks{view}.mp4")
        cmd_tracks = [
            'ffmpeg', 
            '-framerate', '30',
            '-i', os.path.join(temp_dir_tracks_only, 'frame_%06d.jpg'),
            '-c:v', 'libx264',
            '-pix_fmt', 'yuv420p',
            tracks_output,
            '-y'
        ]
        
        for cmd, output_path, name in [
            (cmd_original, original_output, "original"),
            (cmd_overlay, overlay_output, "with tracks"),
            (cmd_tracks, tracks_output, "tracks only")
        ]:
            result = subprocess.run(cmd, capture_output=True, text=True)
            if result.returncode == 0:
                print(f"{name} video saved to: {output_path}")
            else:
                print(f"Failed to create {name} video for view {view}")

In [ ]:
from tqdm import tqdm
import os
import subprocess
import cv2
import numpy as np
import tempfile
import glob

base_path = f"/restricted/projectnb/cs599dg/mwakeham/datasets/panoptic-multiview/{scene}/ims"

i = 99 # example number
example = all_examples[i]
views_to_convert = example['views']
trajectories_pixel = example['trajectories_pixel'] 

visibilities = point_tracks['per_view_visibilities']
selected_points = example['points']
selected_views = example['views']

point_visibilities = visibilities[selected_views][:, :, selected_points]

print(f"Converting views: {views_to_convert}")

fade_frames = 10
pink = (255, 0, 255)
blue = (248, 252, 3)

num_total_points = trajectories_pixel.shape[2]
pink_points_count = int(2 * num_total_points / 3)
point_colors = ['pink'] * pink_points_count + ['blue'] * (num_total_points - pink_points_count)

print(f"Total points: {num_total_points}, Pink: {pink_points_count}, Blue: {num_total_points - pink_points_count}")

# Pre-compute color gradients for faster access
pink_gradients = []
blue_gradients = []
for fade_idx in range(fade_frames):
    ratio = fade_idx / fade_frames
    # Pink gradient
    b = int(255 + (180 - 255) * ratio)
    g = int(0 + (105 - 0) * ratio)
    r = int(255 + (255 - 255) * ratio)
    pink_gradients.append((b, g, r))
    
    # Blue gradient  
    b = int(255 + (180 - 255) * ratio)
    g = int(0 + (0 - 0) * ratio)
    r = int(0 + (0 - 0) * ratio)
    blue_gradients.append((b, g, r))

for view_idx, view in enumerate(views_to_convert):
    image_folder = os.path.join(base_path, str(view))
    output_dir = f"examples_visualization/{scene}/example{i}"
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = sorted(glob.glob(os.path.join(image_folder, "*.jpg")))
    if not image_files:
        print(f"No images found in {image_folder}")
        continue
    
    # Create temporary directories
    with tempfile.TemporaryDirectory() as temp_dir_original, \
         tempfile.TemporaryDirectory() as temp_dir_overlay, \
         tempfile.TemporaryDirectory() as temp_dir_tracks_only, \
         tempfile.TemporaryDirectory() as temp_dir_pink_overlay, \
         tempfile.TemporaryDirectory() as temp_dir_blue_overlay, \
         tempfile.TemporaryDirectory() as temp_dir_pink_only, \
         tempfile.TemporaryDirectory() as temp_dir_blue_only:
        
        view_trajectories = trajectories_pixel[view_idx]
        view_visibilities = point_visibilities[view_idx]
        num_frames = min(len(view_trajectories), len(image_files))
        
        images = []
        for frame_idx, image_path in enumerate(image_files):
            if frame_idx >= num_frames:
                break
            img = cv2.imread(image_path)
            images.append(img)
        
        if not images:
            continue
            
        for frame_idx in tqdm(range(num_frames)):
            if frame_idx >= len(images):
                break
                
            img = images[frame_idx]
            
            cv2.imwrite(os.path.join(temp_dir_original, f"frame_{frame_idx:06d}.jpg"), img)
            
            img_overlay = img.copy()
            img_pink_overlay = img.copy()
            img_blue_overlay = img.copy()
            tracks_only = np.zeros_like(img)
            pink_only = np.zeros_like(img)
            blue_only = np.zeros_like(img)
            
            for point_idx in range(len(view_trajectories[frame_idx])):
                if view_visibilities[frame_idx, point_idx] < 1:
                    continue
                
                current_pos = view_trajectories[frame_idx, point_idx]
                if np.isnan(current_pos[0]) or np.isnan(current_pos[1]):
                    continue
                
                current_x, current_y = int(round(current_pos[0])), int(round(current_pos[1]))
                
                actual_point_idx = selected_points[point_idx] if point_idx < len(selected_points) else point_idx
                color_type = point_colors[actual_point_idx % len(point_colors)]
                
                # start_frame = max(0, frame_idx - fade_frames)
                # for hist_frame in range(start_frame, frame_idx):
                #     if (view_visibilities[hist_frame, point_idx] < 1 or 
                #         view_visibilities[hist_frame + 1, point_idx] < 1):
                #         continue
                    
                #     fade_idx = frame_idx - hist_frame - 1
                #     if fade_idx >= fade_frames:
                #         continue
                    
                #     # Get pre-computed color
                #     if color_type == 'pink':
                #         line_color = pink_gradients[fade_idx]
                #     else:
                #         line_color = blue_gradients[fade_idx]
                    
                #     # Get positions
                #     hist_pos = view_trajectories[hist_frame, point_idx]
                #     next_pos = view_trajectories[hist_frame + 1, point_idx]
                    
                #     if (not np.isnan(hist_pos[0]) and not np.isnan(hist_pos[1]) and 
                #         not np.isnan(next_pos[0]) and not np.isnan(next_pos[1])):
                        
                #         hist_x, hist_y = int(round(hist_pos[0])), int(round(hist_pos[1]))
                #         next_x, next_y = int(round(next_pos[0])), int(round(next_pos[1]))
                        
                #         # Draw lines on appropriate images
                #         # For overlay images with opacity
                #         overlay_temp = img_overlay.copy()
                #         cv2.line(overlay_temp, (hist_x, hist_y), (next_x, next_y), line_color, 1)
                #         cv2.addWeighted(overlay_temp, 0.5, img_overlay, 0.5, 0, img_overlay)

                #         if color_type == 'pink':
                #             overlay_temp = img_pink_overlay.copy()
                #             cv2.line(overlay_temp, (hist_x, hist_y), (next_x, next_y), line_color, 1)
                #             cv2.addWeighted(overlay_temp, 0.5, img_pink_overlay, 0.5, 0, img_pink_overlay)
                #             cv2.line(pink_only, (hist_x, hist_y), (next_x, next_y), line_color, 1)
                #         else:
                #             overlay_temp = img_blue_overlay.copy()
                #             cv2.line(overlay_temp, (hist_x, hist_y), (next_x, next_y), line_color, 1)
                #             cv2.addWeighted(overlay_temp, 0.5, img_blue_overlay, 0.5, 0, img_blue_overlay)
                #             cv2.line(blue_only, (hist_x, hist_y), (next_x, next_y), line_color, 1)
                
                point_color = pink if color_type == 'pink' else blue
                cv2.circle(img_overlay, (current_x, current_y), 1, point_color, -1)
                cv2.circle(tracks_only, (current_x, current_y), 1, point_color, -1)
                
                if color_type == 'pink':
                    cv2.circle(img_pink_overlay, (current_x, current_y), 1, point_color, -1)
                    cv2.circle(pink_only, (current_x, current_y), 1, point_color, -1)
                else:
                    cv2.circle(img_blue_overlay, (current_x, current_y), 1, point_color, -1)
                    cv2.circle(blue_only, (current_x, current_y), 1, point_color, -1)
            
            # Save all frame types
            cv2.imwrite(os.path.join(temp_dir_overlay, f"frame_{frame_idx:06d}.jpg"), img_overlay)
            cv2.imwrite(os.path.join(temp_dir_pink_overlay, f"frame_{frame_idx:06d}.jpg"), img_pink_overlay)
            cv2.imwrite(os.path.join(temp_dir_blue_overlay, f"frame_{frame_idx:06d}.jpg"), img_blue_overlay)
            cv2.imwrite(os.path.join(temp_dir_tracks_only, f"frame_{frame_idx:06d}.jpg"), tracks_only)
            cv2.imwrite(os.path.join(temp_dir_pink_only, f"frame_{frame_idx:06d}.jpg"), pink_only)
            cv2.imwrite(os.path.join(temp_dir_blue_only, f"frame_{frame_idx:06d}.jpg"), blue_only)
        
        video_commands = [
            (['ffmpeg', '-framerate', '30', '-i', os.path.join(temp_dir_original, 'frame_%06d.jpg'),
              '-c:v', 'libx264', '-pix_fmt', 'yuv420p', os.path.join(output_dir, f"view{view}.mp4"), '-y'], "original"),
            
            (['ffmpeg', '-framerate', '30', '-i', os.path.join(temp_dir_overlay, 'frame_%06d.jpg'),
              '-c:v', 'libx264', '-pix_fmt', 'yuv420p', os.path.join(output_dir, f"view{view}_with_tracks.mp4"), '-y'], "with tracks"),
            
            (['ffmpeg', '-framerate', '30', '-i', os.path.join(temp_dir_pink_overlay, 'frame_%06d.jpg'),
              '-c:v', 'libx264', '-pix_fmt', 'yuv420p', os.path.join(output_dir, f"view{view}_pink_overlay.mp4"), '-y'], "pink overlay"),
            
            (['ffmpeg', '-framerate', '30', '-i', os.path.join(temp_dir_blue_overlay, 'frame_%06d.jpg'),
              '-c:v', 'libx264', '-pix_fmt', 'yuv420p', os.path.join(output_dir, f"view{view}_blue_overlay.mp4"), '-y'], "blue overlay"),
            
            (['ffmpeg', '-framerate', '30', '-i', os.path.join(temp_dir_tracks_only, 'frame_%06d.jpg'),
              '-c:v', 'libx264', '-pix_fmt', 'yuv420p', os.path.join(output_dir, f"tracks{view}.mp4"), '-y'], "tracks only"),
            
            (['ffmpeg', '-framerate', '30', '-i', os.path.join(temp_dir_pink_only, 'frame_%06d.jpg'),
              '-c:v', 'libx264', '-pix_fmt', 'yuv420p', os.path.join(output_dir, f"pink_tracks{view}.mp4"), '-y'], "pink tracks only"),
            
            (['ffmpeg', '-framerate', '30', '-i', os.path.join(temp_dir_blue_only, 'frame_%06d.jpg'),
              '-c:v', 'libx264', '-pix_fmt', 'yuv420p', os.path.join(output_dir, f"blue_tracks{view}.mp4"), '-y'], "blue tracks only")
        ]
        
        import concurrent.futures
        def run_ffmpeg(cmd_name):
            cmd, name = cmd_name
            result = subprocess.run(cmd, capture_output=True, text=True)
            if result.returncode == 0:
                return f"{name} video created"
            else:
                return f"Failed to create {name} video"
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(run_ffmpeg, video_commands))
            for result in results:
                print(result)

# colored

### save as gifs

In [ ]:
import os
import subprocess

folder_path = f"examples_visualization/{scene}/example{i}"  # Change this to your folder

for file in os.listdir(folder_path):
    if file.endswith(".mp4"):
        mp4_path = os.path.join(folder_path, file)
        gif_path = os.path.join(folder_path, f"{os.path.splitext(file)[0]}.gif")
        
        cmd = [
            'ffmpeg',
            '-i', mp4_path,
            '-vf', 'fps=30,scale=640:-1:flags=lanczos',
            '-c:v', 'gif',
            gif_path,
            '-y'
        ]
        
        result = subprocess.run(cmd, capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✓ Converted {file} to GIF")
        else:
            print(f"✗ Failed to convert {file}")

### save tracks in structure compatible with trajan

In [58]:
import os
import numpy as np

output_dir = f"/restricted/projectnb/cs599dg/mwakeham/trajectory_examples/{scene}"
os.makedirs(output_dir, exist_ok=True)

for idx, example in enumerate(all_examples, start=1):
    example_dir = os.path.join(output_dir, f"example{idx}")
    os.makedirs(example_dir, exist_ok=True)
    
    traj_pixel = example['trajectories_pixel']
    selected_views = example['views']
    
    visible = (traj_pixel.sum(axis=-1, keepdims=True) != 0).astype(np.float32)
    
    for v_idx, view in enumerate(selected_views):
        view_traj = traj_pixel[v_idx]
        view_vis  = visible[v_idx]
        
        np.save(os.path.join(example_dir, f"tracks_view{v_idx}.npy"), view_traj)
        np.save(os.path.join(example_dir, f"visible_view{v_idx}.npy"), view_vis)